In [ ]:
from misc.preferences import PREFERENCES
from misc.run_configuration import good_germeval_params, default_params, OutputLayerType
from misc.experimental_environment import Experiment
import time
from data.germeval2017 import germeval2017_dataset as dsl
import pprint
from misc import utils
import pandas as pd
import os

STATUS_FAIL = 'fail'
STATUS_OK = 'ok'

In [ ]:
PREFERENCES.defaults(
        data_root='./data/data/germeval2017',
        data_train='train_v1.4.tsv',    
        data_validation='dev_v1.4.tsv',
        data_test='test_TIMESTAMP1.tsv',
        source_index=0,
        target_vocab_index=2,
        file_format='csv',
        language='de'
    )
main_experiment_name = 'GermEval-2017_Experiment'
use_cuda = True

In [ ]:
baseline = good_germeval_params
print(pprint.pformat(baseline, indent=2))

In [ ]:
experiments = [
    {
        'name': 'Baseline - LM-H',
        'description': 'Baseline classification for the GermEval-2017 task using the linear mean head',
        'loss': 1000,
        'f1': -1,
        'rc': {
            'task': 'germeval'
        }
    },
    {
        'name': 'Baseline - CNN-H',
        'description': 'Baseline classification for the GermEval-2017 task using the cnn head',
        'loss': 1000,
        'f1': -1,
        'rc': {
            'task': 'germeval',
            'output_layer_type': OutputLayerType.Convolutions
        }
    },
    {
        'name': 'Attributes',
        'description': 'Classification of only attributes',
        'loss': 1000,
        'f1': -1,
        'rc': {
            'task': 'germeval_multitask'
        }
    }
]

In [ ]:
utils.get_current_git_commit()
print('Current commit: ' + utils.get_current_git_commit())

In [ ]:
for e in experiments:
    name = e['name']
    print(f'#########################################################################\n\nExperiment Name: {name}\n')
    print('#########################################################################\n\n')
    test_params = {**baseline, **{'num_epochs': 35, 'language': 'de'}}
    test_params = {**test_params, **e['rc']}
    e = Experiment(main_experiment_name, e['description'], default_params, test_params, dsl, runs=5)
    df, e_path = e.run()



In [ ]:
p = os.path.join(e_path, 'exp_df.pkl')
print('Save dataframe of experiment to ' + p)
df.to_pickle(p)